In [1]:
#IMPORTAR PACKAGES y LIBRARYS

#Librerías útiles
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import strptime
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn import preprocessing
from itertools import combinations
from dateutil.relativedelta import relativedelta
from tqdm.notebook import tqdm
from pmdarima.utils import diff_inv
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import itertools
from datetime import datetime, timedelta
#otros
import warnings
warnings.filterwarnings("ignore")

#Modelos de Pruebas Estadísticas
from statsmodels.tools.eval_measures import mse,rmse,rmspe,bias,meanabs
import statsmodels.tsa.stattools as ts

#Modelos predictivos estadísticos
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.statespace.varmax import VARMAX
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.api import VAR
from pmdarima import auto_arima
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.diagnostics import performance_metrics
from prophet.diagnostics import cross_validation
from prophet.plot import plot_cross_validation_metric

#Configuraciones
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Importing plotly failed. Interactive plots will not work.


In [2]:
#DICCIONARIOS, OBJETOS
dic_mes = {
        'Enero': 1,
        'Febrero': 2,
        'Marzo': 3,
        'Abril':4,
         'Mayo':5,
         'Junio':6,
         'Julio':7,   
         'Agosto':8,
         'Septiembre':9,
         'Setiembre':9,
         'Octubre':10,
         'Noviembre':11,
         'Diciembre':12
        }


In [3]:
#PREPARACIÓN DATA
df1= pd.read_excel("C:/Renato/script_atenea/consolidado_moderno.xlsx"
,sheet_name=None)
df1 = pd.concat(df1, ignore_index=True)
df2= pd.read_excel("C:/Renato/script_atenea/consolidado_tradicional.xlsx"
,sheet_name=None)
df2 = pd.concat(df2, ignore_index=True)
df3= pd.read_excel("C:/Renato/script_atenea/hiperparameters_tunned.xlsx"
,sheet_name="Hoja1")


df= pd.concat([df1,df2])

df = df.rename(columns={'OFICINA DE VENTAS': 'CANAL'})
df.replace(dic_mes, inplace=True)
df['Quarter'] = df['MES'].apply(lambda x: 'Quarter1' if x in [1, 2, 3] else 'Quarter2' if x in [4, 5, 6] else 'Quarter3' if x in [7, 8, 9] else 'Quarter4')
df['Month_in_Quarter'] = df['MES'].apply(lambda x: 1 if x % 3 == 1 else 2 if x % 3 == 2 else 3)
df["PERIODO"]= df["AÑO"].astype(str)+"-"+df["MES"].astype(str)
df['PERIODO'] = pd.to_datetime(df['PERIODO'])   

df_canal_cat=df[['CANAL','CATEGORIA']].drop_duplicates()
canales =df['CANAL'].unique().tolist()
categorias =df['CATEGORIA'].unique().tolist()

modelos=["MA","Holt Winters","SARIMA","VAR0","VAR1","VAR2","Prophet"]


df_output_atenea=pd.DataFrame(columns=["Canal","Categoría","Modelo","Score","predicción_mes_siguiente"])


In [4]:
#MODELOS
####################
####################

for can in canales:
    for cat in categorias:

        df_1=df[df["CANAL"]==can]
        df_2=df_1[df_1["CATEGORIA"]==cat]
        df_prophet= df_2.copy()
        #df['PERIODO'] = df['PERIODO'].dt.to_period('M')
        df_2 = df_2.set_index('PERIODO')

        cols=['PERIODO','AÑO','MES','Quarter','Month_in_Quarter','OFICINA DE VENTAS'
            ,'VENTA SELL IN', 'VENTA SELL OUT TOTAL'
            , 'SHARE VOLUMEN ALICORP', 'SHARE VOLUMEN MERCADO'
            ,'SHARE % ALICORP','SHARE % MERCADO']

        cols_data =['VENTA SELL IN'
            , 'VENTA SELL OUT TOTAL'
            #, 'SHARE VOLUMEN ALICORP'
            #,'SHARE VOLUMEN MERCADO'
            #,'SHARE % ALICORP'
            #,'SHARE % MERCADO'
            ]

        cols_data_out =['VENTA SELL IN'
            , 'VENTA SELL OUT TOTAL'
            #, 'SHARE VOLUMEN ALICORP'
            #,'SHARE VOLUMEN MERCADO'
            #,'SHARE % ALICORP'
            #,'SHARE % MERCADO'
            ]

        df_ini=df_2.copy()

        df_train= df_ini[cols_data]
        df_outlier=df_ini[cols_data_out]

        #HIPERPARAMETRO 1
        outliers_fraction = float(.06)

        model =  IsolationForest(contamination=outliers_fraction)
        model.fit(df_outlier)
        df_train['anomaly'] = model.predict(df_outlier)
        df_train['VENTA SELL IN'] = df_train.apply(lambda rows: np.NaN if rows["anomaly"] == -1 else rows["VENTA SELL IN"], axis = 1)

        #HIPERPARAMETRO 2
        window_size = 3

        df_train['VENTA SELL IN']=df_train['VENTA SELL IN'].fillna(df_train['VENTA SELL IN'].rolling(window_size+1, min_periods=1).mean())

        default_value = df_train["VENTA SELL IN"].mean()  # You can change this to your desired default value
        df_train['VENTA SELL IN'].fillna(default_value, inplace=True)

        df_train= df_train.drop(columns=["anomaly"])
        Y_prophet_sinoutlier= df_train["VENTA SELL IN"]

        print("---------------------------------------------------------------------")
        #MA
        nobs= 18


        df_pred = df_train.copy()
        df_pred['VENTA SELL IN_t-1'] = df_pred['VENTA SELL IN'].shift()

        # mean of the last 5 values
        df_pred['VENTA SELL IN_pred'] = df_pred['VENTA SELL IN_t-1'].rolling(3).mean()
        df_pred['std_50'] = df_pred['VENTA SELL IN_pred'].rolling(nobs).std()

        # get last 50 weekly sales predictions
        df_pred = df_pred[-nobs:]
        # confidence interval
        df_pred['ci_lower'] = df_pred['VENTA SELL IN_pred']-2*df_pred['std_50']
        df_pred['ci_upper'] = df_pred['VENTA SELL IN_pred']+2*df_pred['std_50']

        MAPE= mean_absolute_percentage_error(df_pred['VENTA SELL IN'], df_pred['VENTA SELL IN_pred'])

        Score= MAPE
        Modelo= "MA"
        prediccion=df_pred["VENTA SELL IN"][-3:].mean() 



        data_x= {"Canal": can, 'Categoría': cat, 'Modelo': Modelo, 'Score': [Score], 'predicción_mes_siguiente': [prediccion]}
        df_output_x= pd.DataFrame(data=data_x)
        df_output_atenea=pd.concat([df_output_atenea,df_output_x],ignore_index=True)
        print(df_output_atenea)


        print("---------------------------------------------------------------------")
        #HOLT WINTERS

        
        df_pred = pd.DataFrame()
        nobs = len(df_train) // 3


        # predict last nobs values
        for i in tqdm(range(nobs)):
            training_data = df_train['VENTA SELL IN'][:(-nobs+i)]
            
            model = ExponentialSmoothing(training_data, initialization_method="estimated",trend='add',
                                        seasonal='add',seasonal_periods=6)
            model_fit = model.fit()
            yhat = model_fit.predict().to_frame()
            df_pred=pd.concat([df_pred,yhat])
        

        df_pred.columns = ['VENTA SELL IN FORECAST']
        df_pred = pd.merge(df_pred, df_train, how = 'left', left_index = True, right_index = True)

        # create confidence interval
        df_pred['std_50'] = df_pred['VENTA SELL IN FORECAST'].rolling(nobs//2).std()
        df_pred['ci_lower'] = df_pred['VENTA SELL IN FORECAST']-2*df_pred['std_50']
        df_pred['ci_upper'] = df_pred['VENTA SELL IN FORECAST']+2*df_pred['std_50']
            
        pd.plotting.register_matplotlib_converters()

        df_pred = df_pred[-nobs//2:]

        MAPE= mean_absolute_percentage_error(df_pred['VENTA SELL IN'], df_pred['VENTA SELL IN FORECAST'])


        Score= MAPE
        Modelo= "Holt Winters"

        training_data = df_train['VENTA SELL IN']
        model = ExponentialSmoothing(training_data, initialization_method="estimated",trend='add',
                                    seasonal='add',seasonal_periods=6)
        model_fit = model.fit()
        yhat = model_fit.predict()

        prediccion= yhat[0]

        data_x= {"Canal": can, 'Categoría': cat, 'Modelo': Modelo, 'Score': [Score], 'predicción_mes_siguiente': [prediccion]}
        df_output_x= pd.DataFrame(data=data_x)
        df_output_atenea=pd.concat([df_output_atenea,df_output_x],ignore_index=True)
        print(df_output_atenea)
        

        print("---------------------------------------------------------------------")

        #MODELO SARIMA
        nobs = len(df_train) // 3

        pred_dict = {'date':[], 'pred_VENTA SELL IN':[], 'ci_lower':[],'ci_upper':[] }

        for i in tqdm(range(nobs)):
            
            # we will use directly monthly sales without transforming
            training_data = df_train['VENTA SELL IN'][:(-nobs+i)]
            model = auto_arima(training_data, start_p=0, start_q=0,
                            max_p=3, max_q=3, max_d = 2,
                            seasonal=True,m=2,
                            start_P=0, start_Q = 0, max_P = 3, max_Q = 3,
                            max_D = 2,information_criterion='aic',test='adf',
                            trace=True)
            #print('best model {}'.format(model.get_params()['order']), end = ' ')
            pred, confint = model.predict(n_periods=1, return_conf_int=True)
            pred_date = training_data.index[-1] + relativedelta(months = 1)
            
            pred_dict['date'].append(pred_date)
            pred_dict['pred_VENTA SELL IN'].append(pred[0])
            pred_dict['ci_lower'].append(confint[0][0])
            pred_dict['ci_upper'].append(confint[0][1])
            

        df_pred = pd.DataFrame(pred_dict)
        df_pred = df_pred.set_index('date')
        df_pred['VENTA SELL IN'] = df_train['VENTA SELL IN'][-nobs:]

        MAPE= mean_absolute_percentage_error(df_pred['VENTA SELL IN'], df_pred['pred_VENTA SELL IN'])

        Score= MAPE
        Modelo= "SARIMA"

        training_data = df_train['VENTA SELL IN']

        model = auto_arima(training_data, start_p=0, start_q=0,
                        max_p=3, max_q=3, max_d = 2,
                        seasonal=True,m=6,
                        start_P=0, start_Q = 0, max_P = 3, max_Q = 3,
                        max_D = 2,information_criterion='aic',test='adf',
                        trace=True)
        print('best model {}'.format(model.get_params()['order']), end = ' ')
        pred, confint = model.predict(n_periods=1, return_conf_int=True)

        prediccion =pred[0]

        data_x= {"Canal": can, 'Categoría': cat, 'Modelo': Modelo, 'Score': [Score], 'predicción_mes_siguiente': [prediccion]}
        df_output_x= pd.DataFrame(data=data_x)
        df_output_atenea=pd.concat([df_output_atenea,df_output_x],ignore_index=True)
        print(df_output_atenea)

        print("---------------------------------------------------------------------")
        #Modelamiento VAR 0

        try:
            df_var = df_train.copy()                
            pred_dict = {'date':[], 'VENTA_SELL_IN_pred':[], 'ci_lower':[],'ci_upper':[] }

            nobs= len(df_var) // 3

            for i in tqdm(range(nobs)):
                
                training_data = df_var[:(-nobs+i)]
                # VAR model selects lag p based on AIC
                model = VAR(training_data)
                model_fit = model.fit(ic = 'aic')
                model_results = model_fit.forecast_interval( y=training_data.to_numpy() , steps=1)

                pred_date = training_data.index[-1] + relativedelta(months = 1)
                pred_dict['date'].append(pred_date)
                pred_dict['VENTA_SELL_IN_pred'].append(model_results[0][0][0])
                pred_dict['ci_lower'].append(model_results[1][0][0])
                pred_dict['ci_upper'].append(model_results[2][0][0])



            # transform data
            df_pred = pd.DataFrame(pred_dict)
            df_pred = df_pred.set_index('date')


            df_pred= pd.concat([df_var,df_pred],axis=1,join="outer")
            n=df_pred.shape[0]
            MAPE= mean_absolute_percentage_error(df_pred['VENTA SELL IN'][-nobs:], df_pred['VENTA_SELL_IN_pred'][-nobs:])

            Modelo= "VAR0"
            Score= MAPE

                
            model = VAR(df_var)
            model_fit = model.fit(ic = 'aic')
            model_results = model_fit.forecast_interval( y=training_data.to_numpy() , steps=1)

            prediccion= model_results[0][0][0]


            data_x= {"Canal": can, 'Categoría': cat, 'Modelo': Modelo, 'Score': [Score], 'predicción_mes_siguiente': [prediccion]}
            df_output_x= pd.DataFrame(data=data_x)
            df_output_atenea=pd.concat([df_output_atenea,df_output_x],ignore_index=True)
            print(df_output_atenea)
        except:
            pass

        print("---------------------------------------------------------------------")
        #VAR1
        try:
            df_var = df_train.copy()
                                                    
            df_1D=df_var.diff().dropna()


            pred_dict = {'date':[], 'VENTA_SELL_IN_1D':[], 'ci_lower':[],'ci_upper':[] }

            nobs= len(df_1D) // 3

            for i in tqdm(range(nobs)):
                
                training_data = df_1D[:(-nobs+i)]
                # VAR model selects lag p based on AIC
                
                model = VAR(training_data)
                model_fit = model.fit(ic = 'aic')
                model_results = model_fit.forecast_interval( y=training_data.to_numpy() , steps=1)

                pred_date = training_data.index[-1] + relativedelta(months = 1)
                pred_dict['date'].append(pred_date)
                pred_dict['VENTA_SELL_IN_1D'].append(model_results[0][0][0])
                pred_dict['ci_lower'].append(model_results[1][0][0])
                pred_dict['ci_upper'].append(model_results[2][0][0])



            # transform data
            df_pred = pd.DataFrame(pred_dict)
            df_pred = df_pred.set_index('date')


            df_pred= pd.concat([df_var,df_pred],axis=1,join="outer")
            #df_pred= df_pred.assign(VENTA_SELL_IN_1D= None)
            df_pred= df_pred.assign(VENTA_SELL_IN_pred= None)


            n=df_pred.shape[0]
            for i in range(2,n):
                df_pred["VENTA_SELL_IN_pred"][i] = df_pred["VENTA SELL IN"].iloc[i-1] + df_pred["VENTA_SELL_IN_1D"][i]

            MAPE= mean_absolute_percentage_error(df_pred['VENTA SELL IN'][-nobs:], df_pred['VENTA_SELL_IN_pred'][-nobs:])


            Modelo= "VAR1"
            Score= MAPE
                
                
                
            model = VAR(df_1D)
            model_fit = model.fit(ic = 'aic')
            model_results = model_fit.forecast_interval( y=training_data.to_numpy() , steps=1)

            pred_date = training_data.index[-1] + relativedelta(months = 1)

            predict_1D=model_results[0][0][0]

            prediccion = df_var["VENTA SELL IN"].iloc[-1] + predict_1D



            data_x= {"Canal": can, 'Categoría': cat, 'Modelo': Modelo, 'Score': [Score], 'predicción_mes_siguiente': [prediccion]}
            df_output_x= pd.DataFrame(data=data_x)
            df_output_atenea=pd.concat([df_output_atenea,df_output_x],ignore_index=True)
            print(df_output_atenea)
        except:
            pass
        print("---------------------------------------------------------------------")
        #VAR2
        try:

            df_var = df_train.copy()
                                                    
            df_1D=df_var.diff().dropna()

            df_2D=df_1D.diff().dropna()

            pred_dict = {'date':[], 'VENTA_SELL_IN_2D':[], 'ci_lower':[],'ci_upper':[] }

            nobs= len(df_2D) // 3

            for i in tqdm(range(nobs)):
                
                training_data = df_2D[:(-nobs+i)]
                
                # VAR model selects lag p based on AIC
                
                model = VAR(training_data)
                model_fit = model.fit(ic = 'aic')
                model_results = model_fit.forecast_interval( y=training_data.to_numpy() , steps=1)

                pred_date = training_data.index[-1] + relativedelta(months = 1)
                pred_dict['date'].append(pred_date)
                pred_dict['VENTA_SELL_IN_2D'].append(model_results[0][0][0])
                pred_dict['ci_lower'].append(model_results[1][0][0])
                pred_dict['ci_upper'].append(model_results[2][0][0])



            # transform data
            df_pred = pd.DataFrame(pred_dict)
            df_pred = df_pred.set_index('date')


            df_pred= pd.concat([df_var,df_pred],axis=1,join="outer")
            df_pred= df_pred.assign(VENTA_SELL_IN_1D= None)
            df_pred= df_pred.assign(VENTA_SELL_IN_pred= None)


            n=df_pred.shape[0]



            for i in range(0,n):
                df_pred["VENTA_SELL_IN_1D"][i] = (df_pred["VENTA SELL IN"].iloc[i-1] - df_pred["VENTA SELL IN"].iloc[i-2]) + df_pred['VENTA_SELL_IN_2D'][i]


            for i in range(0,n):
                df_pred["VENTA_SELL_IN_pred"][i] = df_pred["VENTA SELL IN"].iloc[i-1] + df_pred["VENTA_SELL_IN_1D"][i]


            MAPE= mean_absolute_percentage_error(df_pred['VENTA SELL IN'][-nobs:], df_pred['VENTA_SELL_IN_pred'][-nobs:])


            Modelo= "VAR2"
            Score= MAPE


            training_data = df_2D

            model = VAR(training_data)
            model_fit = model.fit(ic = 'aic')
            model_results = model_fit.forecast_interval( y=training_data.to_numpy() , steps=1)

            predict_2D= model_results[0][0][0]

            predict_1D = (df_pred["VENTA SELL IN"].iloc[1] - df_pred["VENTA SELL IN"].iloc[-2]) + predict_2D
            prediccion = df_pred["VENTA SELL IN"].iloc[-1] + predict_2D



            data_x= {"Canal": can, 'Categoría': cat, 'Modelo': Modelo, 'Score': [Score], 'predicción_mes_siguiente': [prediccion]}
            df_output_x= pd.DataFrame(data=data_x)
            df_output_atenea=pd.concat([df_output_atenea,df_output_x],ignore_index=True)
            print(df_output_atenea)
        except:
            pass

        print("---------------------------------------------------------------------")

        df_prophet["PERIODO"]= df_prophet["AÑO"].astype(str)+"-"+df_prophet["MES"].astype(str)
        df_prophet = df_prophet.rename(columns={'PERIODO': 'ds',
                                'VENTA SELL IN': 'y'})
        cols_data_prophet =['ds','y'
            #, 'VENTA SELL OUT TOTAL'
            #, 'SHARE VOLUMEN ALICORP'
            #,'SHARE VOLUMEN MERCADO'
            #,'SHARE % ALICORP'
            #,'SHARE % MERCADO'
            ]

        df_train=df_prophet[cols_data_prophet]
        df_train["y"]=Y_prophet_sinoutlier.values

        start_date= pd.to_datetime(min(df_train["ds"])) + relativedelta(months=+15)
        end_date= pd.to_datetime(max(df_train["ds"])) + relativedelta(months=-1)


        nobs = int(len(df_train) * 0.3)
        
        cutoffs = pd.date_range(start=start_date, end= end_date, freq='1MS') -timedelta(days=1)
        horizon_var = f"{nobs} days"

        training_data = df_train.copy()
        
        '''param_grid = {  
            'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
            'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
        }

        # Generate all combinations of parameters
        all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
        rmses = []  # Store the RMSEs for each params here

        # Use cross validation to evaluate all parameters
        for params in all_params:
            model = Prophet(**params).fit(training_data)  # Fit model with given params
            df_cv = cross_validation(model,horizon=horizon_var,cutoffs=cutoffs)
            df_p = performance_metrics(df_cv)
            rmses.append(df_p['rmse'].values[0])

        # Find the best parameters
        tuning_results = pd.DataFrame(all_params)
        tuning_results['rmse'] = rmses
        print(tuning_results)

        best_params = all_params[np.argmin(rmses)]
        best_params=list(best_params.items())
        print(best_params) '''
        #best_params[0][1]
        #best_params[1][1]

        df_hiperparameter_filtered= df3[(df3["Canal"]==can)& (df3["Categoría"]==cat) ]
        hiper_1=df_hiperparameter_filtered["changepoint_prior_scale"].iloc[0]
        hiper_2=df_hiperparameter_filtered["seasonality_prior_scale"].iloc[0]

        model_2 = Prophet(changepoint_prior_scale=hiper_1,
                    seasonality_prior_scale= hiper_2).fit(training_data)  
        future = model_2.make_future_dataframe(periods=1,freq="MS")
        forecast = model_2.predict(future)

        df_cv = cross_validation(model_2,horizon=horizon_var,cutoffs=cutoffs)
        df_p = performance_metrics(df_cv)


        Modelo= "Prophet"
        Score=df_p["mape"][0]
        shape_forecast= forecast.shape
        prediccion= forecast["yhat"][shape_forecast[0]-1]


        data_x= {"Canal": can, 'Categoría': cat, 'Modelo': Modelo, 'Score': [Score], 'predicción_mes_siguiente': [prediccion]}
        df_output_x= pd.DataFrame(data=data_x)

        df_output_atenea=pd.concat([df_output_atenea,df_output_x],ignore_index=True)
        print(df_output_atenea)





IsolationForest(contamination=0.06)

---------------------------------------------------------------------
     Canal Categoría Modelo     Score  predicción_mes_siguiente
0  MODERNO   Aceites     MA  0.189683               1408.883298
---------------------------------------------------------------------


  0%|          | 0/19 [00:00<?, ?it/s]

     Canal Categoría        Modelo     Score  predicción_mes_siguiente
0  MODERNO   Aceites            MA  0.189683               1408.883298
1  MODERNO   Aceites  Holt Winters  0.239151               1284.077824
---------------------------------------------------------------------


  0%|          | 0/19 [00:00<?, ?it/s]

...

"param_grid = {  \n            'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],\n            'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],\n        }\n\n        # Generate all combinations of parameters\n        all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]\n        rmses = []  # Store the RMSEs for each params here\n\n        # Use cross validation to evaluate all parameters\n        for params in all_params:\n            model = Prophet(**params).fit(training_data)  # Fit model with given params\n            df_cv = cross_validation(model,horizon=horizon_var,cutoffs=cutoffs)\n            df_p = performance_metrics(df_cv)\n            rmses.append(df_p['rmse'].values[0])\n\n        # Find the best parameters\n        tuning_results = pd.DataFrame(all_params)\n        tuning_results['rmse'] = rmses\n        print(tuning_results)\n\n        best_params = all_params[np.argmin(rmses)]\n        best_params=list(best_params.items())\

15:51:05 - cmdstanpy - INFO - Chain [1] start processing
15:51:05 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/17 [00:00<?, ?it/s]

15:51:05 - cmdstanpy - INFO - Chain [1] start processing
15:51:14 - cmdstanpy - INFO - Chain [1] done processing
15:51:14 - cmdstanpy - INFO - Chain [1] start processing
15:51:22 - cmdstanpy - INFO - Chain [1] done processing
15:51:22 - cmdstanpy - INFO - Chain [1] start processing
15:51:30 - cmdstanpy - INFO - Chain [1] done processing
15:51:30 - cmdstanpy - INFO - Chain [1] start processing
15:51:39 - cmdstanpy - INFO - Chain [1] done processing
15:51:39 - cmdstanpy - INFO - Chain [1] start processing
15:51:47 - cmdstanpy - INFO - Chain [1] done processing
15:51:47 - cmdstanpy - INFO - Chain [1] start processing
15:51:56 - cmdstanpy - INFO - Chain [1] done processing
15:51:56 - cmdstanpy - INFO - Chain [1] start processing
15:52:05 - cmdstanpy - INFO - Chain [1] done processing
15:52:05 - cmdstanpy - INFO - Chain [1] start processing
15:52:05 - cmdstanpy - INFO - Chain [1] done processing
15:52:05 - cmdstanpy - INFO - Chain [1] start processing
15:52:05 - cmdstanpy - INFO - Chain [1]

          Canal Categoría        Modelo     Score  predicción_mes_siguiente
0       MODERNO   Aceites            MA  0.189683               1408.883298
1       MODERNO   Aceites  Holt Winters  0.239151               1284.077824
2       MODERNO   Aceites        SARIMA  0.208551               1536.871678
3       MODERNO   Aceites          VAR0  0.257427               1663.341308
4       MODERNO   Aceites          VAR1  0.184001               1947.198879
..          ...       ...           ...       ...                       ...
57  TRADICIONAL    Lejias        SARIMA  0.067394               2594.524534
58  TRADICIONAL    Lejias          VAR0  0.266977               2655.732046
59  TRADICIONAL    Lejias          VAR1  0.168199               2698.342579
60  TRADICIONAL    Lejias          VAR2  0.213586               3452.140364
61  TRADICIONAL    Lejias       Prophet  0.104208               2543.522342

[62 rows x 5 columns]


In [5]:
#OUTPUT
df_output_atenea
df_output_atenea.to_excel("C:/Renato/script_atenea/Split/Output_x.xlsx"
,sheet_name="Hoja1")

,Canal,Categoría,Modelo,Score,predicción_mes_siguiente
0,MODERNO,Aceites,MA,0.189683,1408.883298
1,MODERNO,Aceites,Holt Winters,0.239151,1284.077824
2,MODERNO,Aceites,SARIMA,0.208551,1536.871678
3,MODERNO,Aceites,VAR0,0.257427,1663.341308
4,MODERNO,Aceites,VAR1,0.184001,1947.198879
...,...,...,...,...,...
57,TRADICIONAL,Lejias,SARIMA,0.067394,2594.524534
58,TRADICIONAL,Lejias,VAR0,0.266977,2655.732046
59,TRADICIONAL,Lejias,VAR1,0.168199,2698.342579
60,TRADICIONAL,Lejias,VAR2,0.213586,3452.140364
